<a href="https://colab.research.google.com/github/othinAbbey/Project-4/blob/main/project_4__part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Import Libraries

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from keras_tuner import RandomSearch


## Step 2: Prepare the Data

In [ ]:
# Load your data here
# For illustration, let's use the Iris dataset
from sklearn.datasets import load_iris

data = load_iris()
X = data.data
y = data.target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Step 3: Build and Compile the Neural Network

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    # Ensure the final layer is appropriate for your task (number of units and activation function).
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()


# Step 4: Fit the Model with Early Stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    # Fit it for 50 epochs
                    epochs=50,
                    # Use a validation_split of .2
                    validation_split=0.2,
                    callbacks=[early_stopping])


# Step 5: Visualize Training History

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# Step 6: Evaluate the Model

In [ ]:
# Evaluate on test data
y_pred = np.argmax(model.predict(X_test), axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Classification Report
print(classification_report(y_test, y_pred, target_names=data.target_names))


# Step 7: Hyperparameter Tuning with Keras Tuner

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from keras_tuner import HyperParameters as hp
import keras_tuner as kt
from keras_tuner import RandomSearch


# Use the Early Stopping callback
# Start with patience =5 monitoring val_accuracy.
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

def build_tuned_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(3, activation='softmax'))  # Adjust according to your task

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = RandomSearch(build_tuned_model,
                     objective='val_accuracy',
                     max_trials=10,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])


Reloading Tuner from my_dir/intro_to_kt/tuner0.json
Epoch 1/50
3/3 [==============================] - 3s 397ms/step - loss: 1.0885 - accuracy: 0.4271 - val_loss: 1.0575 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 0s 82ms/step - loss: 0.9127 - accuracy: 0.6667 - val_loss: 0.9072 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 0s 39ms/step - loss: 0.8459 - accuracy: 0.7292 - val_loss: 0.7831 - val_accuracy: 0.7083
Epoch 4/50
3/3 [==============================] - 0s 80ms/step - loss: 0.7829 - accuracy: 0.6667 - val_loss: 0.7323 - val_accuracy: 0.7083
Epoch 5/50
3/3 [==============================] - 0s 80ms/step - loss: 0.7280 - accuracy: 0.6979 - val_loss: 0.7255 - val_accuracy: 0.6667
Epoch 6/50
3/3 [==============================] - 0s 75ms/step - loss: 0.6793 - accuracy: 0.7188 - val_loss: 0.7567 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 0s 69ms/step - loss: 0.6465 - accuracy: 0.7708 - val_loss: 0.67

# Step 8: Evaluate the Tuned Model

In [ ]:
# Evaluate the tuned model on test data
y_pred = np.argmax(model.predict(X_test), axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Classification Report
print(classification_report(y_test, y_pred, target_names=data.target_names))
